In [1]:
#careful
import pandas as pd #nul
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import json
import geopandas as gpd
from tool_function import *
import altair as alt
from altair import expr, datum
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
#Sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import RocCurveDisplay
from sklearn.base import clone
from sklearn import linear_model
import xgboost

# Data Prep

In [3]:
mutations_init = pd.read_pickle('../../data_mutation/geometric_mutations.pkl')

In [4]:
mutations_p = mutation_process(mutations_init)
mutations_p  = niveau_center_connexion(mutations_p)
mutations_p = pop_commune_year(mutations_p)

hi!


/Users/muhieddineugo/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
# We only want newly built
# mutations = mutations[mutations.libnatmut.isin(["Vente en l'état futur d'achèvement", 'Vente'])].copy()
# REMOVE ALL THESE !! the no surface and no one maison/apt etc; 12% !!
mutations = mutations_init.copy()
print(len(mutations))
df_surf = mutations[~((mutations.sbati ==0) & (mutations[['nblocmai', 'nblocapt', 'nblocact']].sum(axis=1) == 0))]
df_surf = df_surf.copy()
df_surf['smoymai'] = df_surf.sbatmai/df_surf.nblocmai
df_surf['smoyapt'] = df_surf.sbatapt/df_surf.nblocapt
df_surf['smoyact'] = df_surf.sbatact/df_surf.nblocact

df_surf.loc[df_surf['sbati'] == 0, 'sbatmai'] = df_surf.groupby('l_codinsee')['smoymai'].transform(np.nanmedian) * df_surf.nblocmai
df_surf.loc[df_surf['sbati'] == 0, 'sbatapt'] = df_surf.groupby('l_codinsee')['smoyapt'].transform(np.nanmedian) * df_surf.nblocapt
df_surf.loc[df_surf['sbati'] == 0, 'sbatact'] = df_surf.groupby('l_codinsee')['smoyact'].transform(np.nanmedian) * df_surf.nblocact

df_surf.loc[df_surf['sbati'] == 0, 'sbati'] = df_surf[['sbatmai', 'sbatapt', 'sbatact']].sum(axis=1)

df_surf.loc[df_surf['sbati'] == 0, 'sbatmai'] = df_surf.groupby('coddep')['smoymai'].transform(np.nanmedian) * df_surf.nblocmai
df_surf.loc[df_surf['sbati'] == 0, 'sbatapt'] = df_surf.groupby('coddep')['smoyapt'].transform(np.nanmedian) * df_surf.nblocapt
df_surf.loc[df_surf['sbati'] == 0, 'sbatact'] = df_surf.groupby('coddep')['smoyact'].transform(np.nanmedian) * df_surf.nblocact

df_surf.loc[df_surf.sbati == 0, 'sbati'] = df_surf[['sbatmai', 'sbatapt', 'sbatact']].sum(axis=1)

df_surf['smoymai'] = df_surf.sbatmai/df_surf.nblocmai
df_surf['smoyapt'] = df_surf.sbatapt/df_surf.nblocapt
df_surf['smoyact'] = df_surf.sbatact/df_surf.nblocact

mutations = df_surf.copy()


#We only want appartements
# mutations = mutations[mutations.libtypbien.isin(['UN APPARTEMENT', 'DEUX APPARTEMENTS', 'APPARTEMENT INDETERMINE'])].copy()

print(len(mutations))
#'libtypbien' to erase?
#'first_idpar' to erase?
mutations = mutations[
  ['idmutation',
       'datemut', 'month', 'year', 'day',
       'coddep', 'libnatmut', 'vefa', 'valeurfonc',
       'l_codinsee', 'nbpar', 
        'sterr', 'nbvolmut', 'nblocmut',
       'l_idlocmut', 'nblocmai', 'nblocapt', 'nblocdep', 'nblocact',
       'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp', 'nbmai1pp',
       'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati', 'sbatmai',
       'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
       'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp',
       'codtypbien', 'libtypbien', 'centroid',
       'geometry', 'nivcentr', 'smoymai', 'smoyapt', 'smoyact']
       ].copy()

#remove rows with a valeurfonc lower than 5k? 
mutations = mutations[(mutations.valeurfonc > 1000) & ~(mutations.valeurfonc.isna())].copy()
mutations =  mutations[mutations["valeurfonc"] <= 1000000].copy()

print(len(mutations))

mutations = mutations[(mutations.smoyapt > 9)].copy()
print(len(mutations))

1271568


ValueError: cannot reindex from a duplicate axis

# Separate X and Y

In [ ]:
for col in ['month', 'year', 'day', 'coddep', 'l_codinsee']:
    mutations[col] = mutations[col].astype('string')

In [6]:
mutations.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'idmutation', 'idmutinvar', 'idopendata',
       'idnatmut', 'codservch', 'refdoc', 'datemut', 'anneemut', 'moismut',
       'coddep', 'libnatmut', 'vefa', 'valeurfonc', 'nbdispo', 'nblot',
       'nbcomm', 'l_codinsee', 'nbsection', 'l_section', 'nbpar', 'l_idpar',
       'nbparmut', 'l_idparmut', 'nbsuf', 'sterr', 'nbvolmut', 'nblocmut',
       'l_idlocmut', 'nblocmai', 'nblocapt', 'nblocdep', 'nblocact',
       'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp', 'nbmai1pp',
       'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati', 'sbatmai',
       'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
       'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp',
       'codtypbien', 'libtypbien', 'department_code', 'first_idpar',
       'geometry', 'id', 'centroid', 'month', 'year', 'day'],
      dtype='object')

In [46]:
sample_df = mutations.sample(n=200000, random_state=42)
X = sample_df[[
       'month', 'anneemut', 'day',
       'coddep', 'libnatmut', 'vefa',
        'l_codinsee', 'sbati', 'sbatmai',
       'sbatapt', 'nivcentr', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
       'sapt5pp', 'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp',
        'sterr', 'libtypbien', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
       'sapt5pp',  'smoymai', 'smoyapt', 'smoyact',
        'nbpar', 'codtypbien', 'nblocmai', 'nblocapt', 'nblocdep', 'nblocact'
       #  'nbvolmut', 'nblocmut',
       # 'l_idlocmut', 'nblocmai', 'nblocapt', 'nblocdep', 'nblocact',
       # 'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp', 'nbmai1pp',
       # 'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati', 'sbatmai',
       # 'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
       # 'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp',
       # 'codtypbien',
]].copy()
#caref
Y = sample_df[['valeurfonc']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [47]:
ohe = OneHotEncoder(handle_unknown='ignore').fit(X_train)
scalery = MinMaxScaler().fit(y_train)
# scalery = StandardScaler().fit(y_train)
y_train = scalery.transform(y_train)


In [48]:
# clf_lin = RandomForestRegressor(random_state=42) 
model = xgboost.XGBRegressor()
pipeline_lin = make_pipeline(ohe, model)

In [49]:
pipeline_lin.fit(X_train, y_train.ravel())

Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,
                              interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_threshold=64, max_cat_to_onehot=4,
                              max_delta_step=0, max_depth=6, max_leaves=0,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1, predictor='auto',
                              random_state=0, ...))])

In [52]:
pred = pipeline_lin.predict(X_test)
inv_pred = scalery.inverse_transform([pred])

In [53]:
print(mean_absolute_error(inv_pred.ravel(), y_test.valeurfonc))
print(r2_score(inv_pred.ravel(), y_test.valeurfonc))

62993.79300493353
0.4827292593518451


In [28]:
y_test['pred'] = inv_pred.ravel()

In [29]:
y_test

,valeurfonc,pred
684139,303000.0,280274.007136
564998,350000.0,313128.271735
938983,165100.0,154211.811384
1161869,225000.0,216353.729796
624073,255000.0,231018.448342
...,...,...
33627,295000.0,385896.866897
744667,450000.0,438726.202475
1110890,98000.0,96775.930627
472064,202000.0,142459.829894
